## Import Libraries

In [1]:
import sys
sys.path.append('../')

from groof.config import * # IMPORTS ALL DIRECTORIES AND FILE PATHS

from groof.ee_indices import * 
import geemap
from geo import *
from ee_pipeline import *
from shapely.geometry import box
import geopandas as gpd
import ee

## Get ROI

In [2]:
bbox = gpd.read_file(BBOX)
bbox_coords = list(bbox.to_crs(4326).total_bounds)
utm_crs = bbox.estimate_utm_crs()


## Authenticate GEE

In [3]:
ee.Authenticate()
ee.Initialize(project='ee-geoai-medo')

# Get Sentinel-2 Data

In [4]:
def mask_s2_clouds(image):
    # Sentinel-2 QA60 band flags:
    # Bit 10: clouds
    # Bit 11: cirrus
    qa = image.select('QA60')

    # Create bitmasks for clouds and cirrus
    cloud_bit_mask = 1 << 10
    cirrus_bit_mask = 1 << 11

    # Both flags should be set to 0 (i.e., no cloud or cirrus)
    mask = qa.bitwiseAnd(cloud_bit_mask).eq(0).And(
           qa.bitwiseAnd(cirrus_bit_mask).eq(0))

    # Return masked and scaled image
    return image.updateMask(mask).copyProperties(image, ["system:time_start"])

# def percentile_clipping(collection, lower=5, upper=95):
    # Compute lower and upper percentiles
    

def mask_outliers(img):
    low = img.reduce(ee.Reducer.percentile([1]))
    high = img.reduce(ee.Reducer.percentile([95]))
    return img.updateMask(
        img.gte(low).And(img.lte(high))
    )

    # return collection.map(mask_outliers)

In [5]:
geom = ee.Geometry.Rectangle(bbox_coords)

# dsm = ee_read("JAXA/ALOS/AW3D30/V4_1").mean().select('DSM').clip(geom)
ee_s2.update(dict(bands = S2_BANDS))

s2_image = ee_get_s2(ee_s2, geom, start_date='2021-01-01', end_date='2025-12-30', max_cloud=15).map(mask_s2_clouds)
s2_image = s2_image.filter(ee.Filter.calendarRange(3,6, 'month')).map(lambda img: img.divide(10000))

s2_mosaic = s2_image.mosaic().clip(geom)
s2_median = s2_image.median().clip(geom)
# s2 = s2_median.updateMask(s2_median.lte(1)).updateMask(s2_median.gte(0))

il_bldg = ee.FeatureCollection('projects/sat-io/open-datasets/ORNL/USA-STRUCTURES/USA_ST_IL')
il_bldg = il_bldg.filterBounds(geom)

## Compute Spectral Indices

In [6]:
# Built-up Index (BU)
##  NDVI - NDBI
bu_median = bu_ee_s2(s2_median, geom)

# ENHANCED VEGETATION INDEX (EVI)
evi_median = evi_ee_s2(s2_median, geom)

# NORMALIZED DIFFERENCE WATER INDEX
## NIR - SWIR / (NIR + SWIR)
ndwi_median = ndwi_ee_s2(s2_median,geom)

# NDVI
ndvi_median = ndvi_ee_s2(s2_median, geom)

# NDPI
NIR = s2_median.select('B8')
RED = s2_median.select('B4')
SWIR = s2_median.select('B11')

n = NIR.subtract(RED.multiply(ee.Number(0.74)).add(SWIR.multiply(ee.Number(0.26))))
d = NIR.add(RED.multiply(ee.Number(0.74)).add(SWIR.multiply(ee.Number(0.26))))

ndpi_median =n.divide(d)

## Map Spectral Indices and Sentinel-2 Image

In [7]:
evi_minmax = evi_median.reduceRegion(reducer = ee.Reducer.minMax(), geometry= geom, scale =10).getInfo()
evi_minmax


{'EVI_max': 1.2967286982937627, 'EVI_min': -0.7408539549421445}

In [8]:
# Initialize Map
m = geemap.Map()
m.add_basemap('SATELLITE')  # or 'Esri.WorldImagery'
m.centerObject(geom)

# SENTINL-2 RGB
m.addLayer(s2_median, dict(min = 0, max = .3, bands =['B4', 'B3', 'B2']) , 'Sentinel-2 RGB')

# INDICES
m.addLayer(evi_median, dict(min = evi_minmax['EVI_min'], max = evi_minmax['EVI_max'], palette = ['red','yellow', 'green']) , 'EVI Median')
# m.addLayer(bu_median, dict(min = -1, max = 1, palette = ['blue','yellow', 'red']) , 'BU Median')
# m.addLayer(ndwi_median, dict(min = -1, max = 1, palette = ['red','yellow', 'blue']) , 'NDWI Median')
m.addLayer(ndvi_median, dict(min = -1, max = 1, palette = ['red','yellow', 'green']) , 'NDVI Median')
m.addLayer(ndpi_median, dict(min = -1, max = 1, palette = ['red','yellow', 'green']) , 'NDPI Median')
# Add Buildings Layer
# m.addLayer(il_bldg, dict(color = 'red', fill = None), 'Building Footprints')
m

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

# Download Layers

### Sentinel-2

In [ ]:
geemap.download_ee_image(
    s2_median,
    filename=S2_MEDIAN,
    region =geom,
    crs=str(utm_crs),
    scale= 10
)

### Spectral Indices

In [ ]:

geemap.download_ee_image(
    ndvi_median,
    filename=NDVI_MEDIAN,
    region =geom,
    crs=str(utm_crs),
    scale= 10
)


geemap.download_ee_image(
    evi_median,
    filename=EVI_MEDIAN,
    region =geom,
    crs=str(utm_crs),
    scale= 10,
    dtype='float32'
)

geemap.download_ee_image(
    bu_median,
    filename=BU_MEDIAN,
    region =geom,
    crs=str(utm_crs),
    scale= 10
)


geemap.download_ee_image(
    ndwi_median,
    filename=NDWI_MEDIAN,
    region =geom,
    crs=str(utm_crs),
    scale= 10
)


geemap.download_ee_image(
    ndpi_median,
    filename=NDPI_MEDIAN,
    region =geom,
    crs=str(utm_crs),
    scale= 10
)


ndpi_median.tif: |          | 0.00/2.04M (raw) [  0.0%] in 00:00 (eta:     ?)

There is no STAC entry for: None


###  Buildings Layer

In [11]:
# geemap.ee_to_geojson(il_bldg, filename=BLDG_FP)